In [12]:
! pip install scikit-image
! pip install filterpy
! pip install pandas
! pip install moviepy

^C
ERROR: Operation cancelled by user


## 동영상 이름

In [2]:
video_path = 'mevid.mp4'
#video_path = '../OUTPUT/backgone.avi'

# nothing

In [2]:
import numpy as np
import cv2 as cv
import sys

def construct_yolo_v3():
    f=open('coco_names.txt', 'r')
    class_names=[line.strip() for line in f.readlines()]

    model=cv.dnn.readNet('yolov3.weights','yolov3.cfg')
    layer_names=model.getLayerNames()
    out_layers=[layer_names[i-1] for i in model.getUnconnectedOutLayers()]
    
    return model,out_layers,class_names

def yolo_detect(img,yolo_model,out_layers):
    height,width=img.shape[0],img.shape[1]
    test_img=cv.dnn.blobFromImage(img,1.0/256,(448,448),(0,0,0),swapRB=True)
    
    yolo_model.setInput(test_img)
    output3=yolo_model.forward(out_layers)
    
    box,conf,id=[],[],[]		# 박스, 신뢰도, 부류 번호
    for output in output3:
        for vec85 in output:
            scores=vec85[5:]
            class_id=np.argmax(scores)
            confidence=scores[class_id]
            if confidence>0.5:	# 신뢰도가 50% 이상인 경우만 취함
                centerx,centery=int(vec85[0]*width),int(vec85[1]*height)
                w,h=int(vec85[2]*width),int(vec85[3]*height)
                x,y=int(centerx-w/2),int(centery-h/2)
                box.append([x,y,x+w,y+h])
                conf.append(float(confidence))
                id.append(class_id)
            
    ind=cv.dnn.NMSBoxes(box,conf,0.5,0.4)
    objects=[box[i]+[conf[i]]+[id[i]] for i in range(len(box)) if i in ind]
    return objects

model,out_layers,class_names=construct_yolo_v3()	# YOLO 모델 생성
colors=np.random.uniform(0,255,size=(100,3))		# 100개 색으로 트랙 구분

from sort import Sort

sort=Sort()

cap=cv.VideoCapture(0)
if not cap.isOpened(): sys.exit('카메라 연결 실패')

while True:
    ret,frame=cap.read()
    if not ret: sys.exit('프레임 획득에 실패하여 루프를 나갑니다.')
        
    res=yolo_detect(frame,model,out_layers)   
    persons=[res[i] for i in range(len(res)) if res[i][5]==0] # 부류 0은 사람

    if len(persons)==0: 
        tracks=sort.update()
    else:
        tracks=sort.update(np.array(persons))
    
    for i in range(len(tracks)):
        x1,y1,x2,y2,track_id=tracks[i].astype(int)
        cv.rectangle(frame,(x1,y1),(x2,y2),colors[track_id],2)
        cv.putText(frame,str(track_id),(x1+10,y1+40),cv.FONT_HERSHEY_PLAIN,3,colors[track_id],2)            
    
    cv.imshow('Person tracking by SORT',frame)
    
    key=cv.waitKey(1) 
    if key==ord('q'): break 
    
cap.release()		# 카메라와 연결을 끊음
cv.destroyAllWindows()

# 동영상 미리보기

In [ ]:
import numpy as np
import cv2 as cv
import sys

def construct_yolo_v3():
    f=open('coco_names.txt', 'r')
    class_names=[line.strip() for line in f.readlines()]

    model=cv.dnn.readNet('yolov3.weights','yolov3.cfg')
    layer_names=model.getLayerNames()
    out_layers=[layer_names[i-1] for i in model.getUnconnectedOutLayers()]
    
    return model,out_layers,class_names

def yolo_detect(img,yolo_model,out_layers):
    height,width=img.shape[0],img.shape[1]
    test_img=cv.dnn.blobFromImage(img,1.0/256,(448,448),(0,0,0),swapRB=True)
    
    yolo_model.setInput(test_img)
    output3=yolo_model.forward(out_layers)
    
    box,conf,id=[],[],[]		# 박스, 신뢰도, 부류 번호
    for output in output3:
        for vec85 in output:
            scores=vec85[5:]
            class_id=np.argmax(scores)
            confidence=scores[class_id]
            if confidence>0.5:	# 신뢰도가 50% 이상인 경우만 취함
                centerx,centery=int(vec85[0]*width),int(vec85[1]*height)
                w,h=int(vec85[2]*width),int(vec85[3]*height)
                x,y=int(centerx-w/2),int(centery-h/2)
                box.append([x,y,x+w,y+h])
                conf.append(float(confidence))
                id.append(class_id)
            
    ind=cv.dnn.NMSBoxes(box,conf,0.5,0.4)
    objects=[box[i]+[conf[i]]+[id[i]] for i in range(len(box)) if i in ind]
    return objects

model,out_layers,class_names=construct_yolo_v3()	# YOLO 모델 생성
colors=np.random.uniform(0,255,size=(100,3))		# 100개 색으로 트랙 구분

from sort import Sort
sort=Sort()
try: 
    #cap = cv.VideoCapture(0) # cam mode
    cap = cv.VideoCapture(video_path) # video file mode
    print("file")
except:
    print("cam")
    cap=cv.VideoCapture(0,cv.CAP_DSHOW)


if not cap.isOpened(): sys.exit('카메라 연결 실패')

while True:
    ret,frame=cap.read()
    if not ret: sys.exit('프레임 획득에 실패하여 루프를 나갑니다.')
        
    res=yolo_detect(frame,model,out_layers)   
    persons=[res[i] for i in range(len(res)) if res[i][5]==0] # 부류 0은 사람

    if len(persons)==0: 
        tracks=sort.update()
    else:
        tracks=sort.update(np.array(persons))
    
    for i in range(len(tracks)):
        x1,y1,x2,y2,track_id=tracks[i].astype(int)
        cv.rectangle(frame,(x1,y1),(x2,y2),colors[track_id],2)
        cv.putText(frame,str(track_id),(x1+10,y1+40),cv.FONT_HERSHEY_PLAIN,3,colors[track_id],2)            
    
    cv.imshow('Person tracking by SORT',frame)
    
    key=cv.waitKey(1) 
    if key==ord('q'): break 
    
cap.release()		# 카메라와 연결을 끊음
cv.destroyAllWindows()

file


# txt 저장

In [ ]:
import numpy as np
import cv2 as cv
import sys

def construct_yolo_v3():
    f=open('coco_names.txt', 'r')
    class_names=[line.strip() for line in f.readlines()]

    model=cv.dnn.readNet('yolov3.weights','yolov3.cfg')
    layer_names=model.getLayerNames()
    out_layers=[layer_names[i-1] for i in model.getUnconnectedOutLayers()]
    
    return model,out_layers,class_names

def yolo_detect(img,yolo_model,out_layers):
    height,width=img.shape[0],img.shape[1]
    test_img=cv.dnn.blobFromImage(img,1.0/256,(448,448),(0,0,0),swapRB=True)
    
    yolo_model.setInput(test_img)
    output3=yolo_model.forward(out_layers)
    
    box,conf,id=[],[],[]		# 박스, 신뢰도, 부류 번호
    for output in output3:
        for vec85 in output:
            scores=vec85[5:]
            class_id=np.argmax(scores)
            confidence=scores[class_id]
            if confidence>0.5:	# 신뢰도가 50% 이상인 경우만 취함
                centerx,centery=int(vec85[0]*width),int(vec85[1]*height)
                w,h=int(vec85[2]*width),int(vec85[3]*height)
                x,y=int(centerx-w/2),int(centery-h/2)
                box.append([x,y,x+w,y+h])
                conf.append(float(confidence))
                id.append(class_id)
            
    ind=cv.dnn.NMSBoxes(box,conf,0.5,0.4)
    objects=[box[i]+[conf[i]]+[id[i]] for i in range(len(box)) if i in ind]
    return objects

model,out_layers,class_names=construct_yolo_v3()	# YOLO 모델 생성
colors=np.random.uniform(0,255,size=(100,3))		# 100개 색으로 트랙 구분

model, out_layers, class_names = construct_yolo_v3()
colors = np.random.uniform(0, 255, size=(100, 3))

from sort import Sort
sort = Sort()
cap = cv.VideoCapture(video_path)

if not cap.isOpened(): 
    sys.exit('동영상 파일을 열 수 없습니다.')

# 결과를 저장할 파일 열기
output_file = open('tracking_results.txt', 'w')

frame_number = 0
while True:
    ret, frame = cap.read()
    if not ret: 
        break
    
    res = yolo_detect(frame, model, out_layers)   
    persons = [res[i] for i in range(len(res)) if res[i][5] == 0]

    if len(persons) == 0: 
        tracks = sort.update()
    else:
        tracks = sort.update(np.array(persons))
    
    for track in tracks:
        x1, y1, x2, y2, track_id = track.astype(int)
        # 파일에 프레임 번호와 바운딩 박스 정보 기록
        output_file.write(f'Frame {frame_number}: ID {track_id} - Box [{x1}, {y1}, {x2}, {y2}]\n')

    frame_number += 1

    key = cv.waitKey(1) 
    if key == ord('q'): 
        break 

cap.release()
output_file.close()
cv.destroyAllWindows()


# CSV 저장

In [8]:
import numpy as np
import cv2 as cv
import sys
import pandas as pd

def construct_yolo_v3():
    f = open('coco_names.txt', 'r')
    class_names = [line.strip() for line in f.readlines()]

    model = cv.dnn.readNet('yolov3.weights', 'yolov3.cfg')
    layer_names = model.getLayerNames()
    out_layers = [layer_names[i - 1] for i in model.getUnconnectedOutLayers()]
    
    return model, out_layers, class_names

def yolo_detect(img, yolo_model, out_layers):
    height, width = img.shape[0], img.shape[1]
    test_img = cv.dnn.blobFromImage(img, 1.0 / 256, (448, 448), (0, 0, 0), swapRB=True)
    
    yolo_model.setInput(test_img)
    output3 = yolo_model.forward(out_layers)
    
    box, conf, id = [], [], []  # 박스, 신뢰도, 부류 번호
    for output in output3:
        for vec85 in output:
            scores = vec85[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # 신뢰도가 50% 이상인 경우만 취함
                centerx, centery = int(vec85[0] * width), int(vec85[1] * height)
                w, h = int(vec85[2] * width), int(vec85[3] * height)
                x, y = int(centerx - w / 2), int(centery - h / 2)
                box.append([x, y, x + w, y + h])
                conf.append(float(confidence))
                id.append(class_id)
            
    ind = cv.dnn.NMSBoxes(box, conf, 0.5, 0.4)
    objects = [box[i] + [conf[i]] + [id[i]] for i in range(len(box)) if i in ind]
    return objects

model, out_layers, class_names = construct_yolo_v3()
colors = np.random.uniform(0, 255, size=(100, 3))

from sort import Sort
sort = Sort()
cap = cv.VideoCapture(video_path)

if not cap.isOpened(): 
    sys.exit('동영상 파일을 열 수 없습니다.')

# 결과를 저장할 리스트 생성
tracking_results = []

frame_number = 0
while True:
    ret, frame = cap.read()
    if not ret: 
        break
    
    res = yolo_detect(frame, model, out_layers)   
    persons = [res[i] for i in range(len(res)) if res[i][5] == 0]

    if len(persons) == 0: 
        tracks = sort.update()
    else:
        tracks = sort.update(np.array(persons))
    
    for track in tracks:
        x1, y1, x2, y2, track_id = track.astype(int)
        # 리스트에 결과 추가
        tracking_results.append({'Frame': frame_number, 'ID': track_id, 'Box': [x1, y1, x2, y2]})

    frame_number += 1

    key = cv.waitKey(1) 
    if key == ord('q'): 
        break 

cap.release()
cv.destroyAllWindows()

# 결과를 DataFrame으로 변환하고 CSV 파일로 저장
tracking_results_df = pd.DataFrame(tracking_results)
tracking_results_df.to_csv('tracking_results_mevid.csv', index=False)

# 동영상 + 텍스트 결과 보기 (csv)

In [10]:
import cv2 as cv
import numpy as np
import pandas as pd

def load_tracking_data(file_path):
    tracking_df = pd.read_csv(file_path)
    tracking_data = {}

    for _, row in tracking_df.iterrows():
        frame_number = row['Frame']
        track_id = row['ID']
        box = eval(row['Box'])  # 문자열 형태의 리스트를 실제 리스트 객체로 변환

        if frame_number not in tracking_data:
            tracking_data[frame_number] = []
        tracking_data[frame_number].append((track_id, box))
    
    return tracking_data


# Load tracking data
tracking_data_path = 'tracking_results_mevid.csv'
tracking_data = load_tracking_data(tracking_data_path)

cap = cv.VideoCapture(video_path)
if not cap.isOpened():
    sys.exit('동영상 파일을 열 수 없습니다.')

# Define the codec and create VideoWriter object
#mode = 'avi'
mode = 'mp4'
if mode == 'avi':
    fourcc = cv.VideoWriter_fourcc(*'XVID')
    out = cv.VideoWriter('output.avi', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))
elif mode == 'mp4':
    fourcc = cv.VideoWriter_fourcc(*'mp4v')
    out = cv.VideoWriter('output2.mp4', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

frame_number = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    if frame_number in tracking_data:
        for track_id, box in tracking_data[frame_number]:
            x1, y1, x2, y2 = box
            cv.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv.putText(frame, str(track_id), (x1, y1 - 10), cv.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Write the frame into the file 'output.avi'
    out.write(frame)

    cv.imshow('Tracking', frame)
    frame_number += 1

    if cv.waitKey(1) & 0xFF == ord('q'):
        break

# Release everything if job is finished
cap.release()
out.release()
cv.destroyAllWindows()


# cut

In [ ]:
import cv2
import numpy as np
from collections import defaultdict

# Function to parse a line from the tracking results and extract frame number, track ID, and bounding box coordinates
def parse_tracking_line(line):
    match = re.match(r'Frame (\d+): ID (\d+) - Box \[(\d+), (\d+), (\d+), (\d+)\]', line)
    if match:
        frame, track_id, x1, y1, x2, y2 = map(int, match.groups())
        return frame, track_id, (x1, y1, x2, y2)
    return None

def parse_tracking_line(line):  
    # Splitting the string to extract the relevant parts
    parts = line.split()

    # Extracting frame and track_id
    frame = int(parts[1].rstrip(':'))
    track_id = int(parts[3])

    # Extracting coordinates
    coordinates_str = input_str.split('[')[1].split(']')[0]
    coordinates = tuple(map(int, coordinates_str.split(',')))

    # Returning the extracted information
    return frame, track_id, coordinates




############## fix

def extract_info_from_file(file_path):

    def extract_info(input_str):
        # Splitting the string to extract the relevant parts
        parts = input_str.split()

        # Extracting frame and track_id
        frame = int(parts[1].rstrip(':'))
        track_id = int(parts[3])

        # Extracting coordinates
        coordinates_str = input_str.split('[')[1].split(']')[0]
        coordinates = tuple(map(int, coordinates_str.split(',')))

        # Returning the extracted information
        return frame, track_id, coordinates

    # Reading the file and processing each line
    with open(file_path, 'r') as file:
        lines = file.readlines()
        return [extract_info(line.strip()) for line in lines]

# Path to the uploaded file
file_path = 'tracking_results.txt'

# Extracting information from the file



##############




# Parsing the tracking data
#tracking_info = parse_tracking_line('tracking_results.txt')


#####
extracted_data = extract_info_from_file(file_path)
tracking_info = extracted_data

####


# Opening the video file
video = cv2.VideoCapture(video_path)

# Check if video opened successfully
if not video.isOpened():
    raise IOError("Error opening video file")

# Video parameters
frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_rate = video.get(cv2.CAP_PROP_FPS)

# Dictionary to store video writers for each track ID
output_videos = defaultdict(lambda: None)

# Process video frame by frame
current_frame = 0
while True:
    ret, frame = video.read()
    if not ret:
        break

    # Process tracking data for the current frame
    frame_tracks = [info for info in tracking_info if info[0] == current_frame]
    for _, track_id, (x1, y1, x2, y2) in frame_tracks:
        # Convert to square bounding box
        width = x2 - x1
        height = y2 - y1
        side_length = max(width, height)
        center_x, center_y = x1 + width // 2, y1 + height // 2
        new_x1 = max(center_x - side_length // 2, 0)
        new_y1 = max(center_y - side_length // 2, 0)
        new_x2 = min(center_x + side_length // 2, frame_width)
        new_y2 = min(center_y + side_length // 2, frame_height)

        # Extract the bounding box
        bbox_frame = frame[new_y1:new_y2, new_x1:new_x2]

        # Check if video writer is already created for this track ID
        if output_videos[track_id] is None:
            output_path = f'/mnt/data/track_{track_id}.avi'
            output_videos[track_id] = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'XVID'), frame_rate, (side_length, side_length))

        # Write the frame to the output video
        output_videos[track_id].write(bbox_frame)

    current_frame += 1

# Release resources
video.release()
for writer in output_videos.values():
    if writer:
        writer.release()

# Provide paths for the user to download
output_video_files = [f'track_{track_id}.avi' for track_id in output_videos.keys()]
output_video_files


['track_2.avi',
 'track_1.avi',
 'track_3.avi',
 'track_5.avi',
 'track_6.avi',
 'track_7.avi',
 'track_10.avi',
 'track_12.avi',
 'track_11.avi',
 'track_13.avi',
 'track_16.avi',
 'track_17.avi',
 'track_18.avi',
 'track_19.avi',
 'track_20.avi',
 'track_23.avi',
 'track_24.avi',
 'track_26.avi',
 'track_27.avi',
 'track_28.avi',
 'track_30.avi',
 'track_31.avi',
 'track_32.avi',
 'track_33.avi',
 'track_34.avi',
 'track_35.avi',
 'track_36.avi',
 'track_37.avi',
 'track_38.avi',
 'track_39.avi',
 'track_41.avi',
 'track_43.avi',
 'track_46.avi',
 'track_48.avi',
 'track_49.avi',
 'track_50.avi',
 'track_51.avi',
 'track_52.avi',
 'track_53.avi',
 'track_54.avi',
 'track_58.avi',
 'track_60.avi',
 'track_62.avi',
 'track_66.avi',
 'track_67.avi',
 'track_68.avi',
 'track_69.avi',
 'track_70.avi',
 'track_74.avi',
 'track_76.avi',
 'track_75.avi',
 'track_77.avi',
 'track_78.avi',
 'track_80.avi',
 'track_79.avi',
 'track_81.avi',
 'track_82.avi',
 'track_87.avi',
 'track_88.avi',
 't

In [ ]:
import cv2
import numpy as np
from collections import defaultdict

# Function to parse a line from the tracking results and extract frame number, track ID, and bounding box coordinates
def parse_tracking_line(line):
    match = re.match(r'Frame (\d+): ID (\d+) - Box \[(\d+), (\d+), (\d+), (\d+)\]', line)
    if match:
        frame, track_id, x1, y1, x2, y2 = map(int, match.groups())
        return frame, track_id, (x1, y1, x2, y2)
    return None

def parse_tracking_line(line):  
    # Splitting the string to extract the relevant parts
    parts = line.split()

    # Extracting frame and track_id
    frame = int(parts[1].rstrip(':'))
    track_id = int(parts[3])

    # Extracting coordinates
    coordinates_str = input_str.split('[')[1].split(']')[0]
    coordinates = tuple(map(int, coordinates_str.split(',')))

    # Returning the extracted information
    return frame, track_id, coordinates




############## fix

def extract_info_from_file(file_path):

    def extract_info(input_str):
        # Splitting the string to extract the relevant parts
        parts = input_str.split()

        # Extracting frame and track_id
        frame = int(parts[1].rstrip(':'))
        track_id = int(parts[3])

        # Extracting coordinates
        coordinates_str = input_str.split('[')[1].split(']')[0]
        coordinates = tuple(map(int, coordinates_str.split(',')))

        # Returning the extracted information
        return frame, track_id, coordinates

    # Reading the file and processing each line
    with open(file_path, 'r') as file:
        lines = file.readlines()
        return [extract_info(line.strip()) for line in lines]

# Path to the uploaded file
file_path = 'tracking_results.txt'

# Extracting information from the file



##############




# Parsing the tracking data
#tracking_info = parse_tracking_line('tracking_results.txt')


#####
extracted_data = extract_info_from_file(file_path)
tracking_info = extracted_data

####


# Opening the video file
video = cv2.VideoCapture(video_path)

# Check if video opened successfully
if not video.isOpened():
    raise IOError("Error opening video file")

# Video parameters
frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_rate = video.get(cv2.CAP_PROP_FPS)

# Dictionary to store video writers for each track ID
output_videos = defaultdict(lambda: None)

# Process video frame by frame
current_frame = 0
while True:
    ret, frame = video.read()
    if not ret:
        break

    # Process tracking data for the current frame
    frame_tracks = [info for info in tracking_info if info[0] == current_frame]
    for _, track_id, (x1, y1, x2, y2) in frame_tracks:
        # Convert to square bounding box
        width = x2 - x1
        height = y2 - y1
        side_length = max(width, height)
        center_x, center_y = x1 + width // 2, y1 + height // 2
        new_x1 = max(center_x - side_length // 2, 0)
        new_y1 = max(center_y - side_length // 2, 0)
        new_x2 = min(center_x + side_length // 2, frame_width)
        new_y2 = min(center_y + side_length // 2, frame_height)

        # Extract the bounding box
        bbox_frame = frame[new_y1:new_y2, new_x1:new_x2]

        # Check if video writer is already created for this track ID
        if output_videos[track_id] is None:
            output_path = f'/mnt/data/track_{track_id}.avi'
            output_videos[track_id] = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'XVID'), frame_rate, (side_length, side_length))

        # Write the frame to the output video
        output_videos[track_id].write(bbox_frame)

    current_frame += 1

# Release resources
video.release()
for writer in output_videos.values():
    if writer:
        writer.release()

# Provide paths for the user to download
output_video_files = [f'track_{track_id}.avi' for track_id in output_videos.keys()]
output_video_files


In [ ]:
import cv2
import os

# Assuming output_videos is a dictionary with track_id as keys and video frames as values
output_videos = {}  # Populate this with your actual video data

# Directory where videos will be saved
output_directory = 'path/to/your/output/folder'  # Replace with the actual path

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Function to save each video
def save_video(track_id, frames, output_dir):
    # Define the path for the output video
    output_path = os.path.join(output_dir, f'track_{track_id}.avi')

    # Define video writer
    # Assuming all frames have the same size and type, use the properties of the first frame
    height, width, layers = frames[0].shape
    video_writer = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'DIVX'), 30, (width, height))

    # Write frames to the video
    for frame in frames:
        video_writer.write(frame)

    # Release the video writer
    video_writer.release()

# Iterate over the output videos and save them
for track_id, frames in output_videos.items():
    save_video(track_id, frames, output_directory)

# Provide paths for the user to download
output_video_files = [os.path.join(output_directory, f'track_{track_id}.avi') for track_id in output_videos.keys()]


In [ ]:
import cv2
import os

# Assuming output_videos is a dictionary with track_id as keys and video frames as values
output_videos = output_video_files  # Populate this with your actual video data

# Directory where videos will be saved
output_directory = 'ouput'  # Replace with the actual path

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Function to save each video
def save_video(track_id, frames, output_dir):
    # Define the path for the output video
    output_path = os.path.join(output_dir, f'track_{track_id}.avi')

    # Define video writer
    # Assuming all frames have the same size and type, use the properties of the first frame
    height, width, layers = frames[0].shape
    video_writer = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'DIVX'), 30, (width, height))

    # Write frames to the video
    for frame in frames:
        video_writer.write(frame)

    # Release the video writer
    video_writer.release()

# Iterate over the output videos and save them
for track_id, frames in output_videos.items():
    save_video(track_id, frames, output_directory)

# Provide paths for the user to download
output_video_files = [os.path.join(output_directory, f'track_{track_id}.avi') for track_id in output_videos.keys()]

AttributeError: 'list' object has no attribute 'items'

# crop video csv

In [ ]:
import cv2
import pandas as pd
import os

def load_tracking_data(csv_path):
    tracking_df = pd.read_csv(csv_path)
    tracking_data = {}
    for _, row in tracking_df.iterrows():
        frame = row['Frame']
        track_id = row['ID']
        box = eval(row['Box'])
        if frame not in tracking_data:
            tracking_data[frame] = []
        tracking_data[frame].append((track_id, box))
    return tracking_data

def save_cropped_video(video_path, tracking_data, output_dir):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise IOError("Error opening video file")
    
    frame_number = 0
    video_writers = {}

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_number in tracking_data:
            for track_id, (x1, y1, x2, y2) in tracking_data[frame_number]:
                cropped_frame = frame[y1:y2, x1:x2]
                ### save as avi
                # if track_id not in video_writers: 
                #     frame_height, frame_width = cropped_frame.shape[:2]
                #     output_path = os.path.join(output_dir, f'track_{track_id}.avi')
                #     video_writers[track_id] = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'XVID'), cap.get(cv2.CAP_PROP_FPS), (frame_width, frame_height))
                if track_id not in video_writers:
                    frame_height, frame_width = cropped_frame.shape[:2]
                    output_path = os.path.join(output_dir, f'track_{track_id}.mp4')
                    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Change here for MP4
                    video_writers[track_id] = cv2.VideoWriter(output_path, fourcc, cap.get(cv2.CAP_PROP_FPS), (frame_width, frame_height))

                video_writers[track_id].write(cropped_frame)

                video_writers[track_id].write(cropped_frame)

        frame_number += 1

    cap.release()
    for writer in video_writers.values():
        writer.release()

# Paths
csv_path = 'tracking_results.csv'
output_dir = 'cropped_videos'

# Make sure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Process the video
tracking_data = load_tracking_data(csv_path)
save_cropped_video(video_path, tracking_data, output_dir)

print("Cropping complete. Videos saved in", output_dir)


Cropping complete. Videos saved in cropped_videos


# crop video again

In [ ]:
import cv2
import pandas as pd
import os

def load_tracking_data(csv_path):
    tracking_df = pd.read_csv(csv_path)
    tracking_data = {}
    for _, row in tracking_df.iterrows():
        frame = row['Frame']
        track_id = row['ID']
        box = eval(row['Box'])  # Convert string representation of list to actual list
        if frame not in tracking_data:
            tracking_data[frame] = []
        tracking_data[frame].append((track_id, box))
    return tracking_data

def save_cropped_videos(video_path, tracking_data, output_dir):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise IOError("Error opening video file")
    
    frame_number = 0
    video_writers = {}
    fps = cap.get(cv2.CAP_PROP_FPS)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_number in tracking_data:
            for track_id, (x1, y1, x2, y2) in tracking_data[frame_number]:
                cropped_frame = frame[y1:y2, x1:x2]

                if track_id not in video_writers:
                    height, width = cropped_frame.shape[:2]
                    output_path = os.path.join(output_dir, f'track_{track_id}.mp4')
                    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # MP4 codec
                    video_writers[track_id] = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

                video_writers[track_id].write(cropped_frame)

        frame_number += 1

    cap.release()
    for writer in video_writers.values():
        writer.release()

# Paths
csv_path = 'tracking_results.csv'
output_dir = 'cropped_videos'

# Make sure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Load tracking data and process the video
tracking_data = load_tracking_data(csv_path)
save_cropped_videos(video_path, tracking_data, output_dir)

print("Cropping complete. Videos saved in", output_dir)


Cropping complete. Videos saved in cropped_videos


# re re video crop

In [ ]:
import cv2
import pandas as pd
import os

def load_tracking_data(csv_path):
    return pd.read_csv(csv_path)

def crop_square(frame, box):
    x, y, x2, y2 = box
    height = y2 - y
    center_x = x + (x2 - x) // 2
    x1 = max(center_x - height // 2, 0)
    x2 = min(center_x + height // 2, frame.shape[1])
    return frame[y:y2, x1:x2]

def save_cropped_videos(video_path, tracking_df, output_dir, fps=30):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise IOError("Error opening video file")
    
    video_writers = {}
    for _, row in tracking_df.iterrows():
        frame_number = row['Frame']
        track_id = row['ID']
        box = eval(row['Box'])
        
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
        ret, frame = cap.read()
        if not ret:
            continue
        
        cropped_frame = crop_square(frame, box)
        height, width = cropped_frame.shape[:2]

        if track_id not in video_writers:
            output_path = os.path.join(output_dir, f'track_{track_id}.mp4')
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            video_writers[track_id] = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

        video_writers[track_id].write(cropped_frame)

    cap.release()
    for writer in video_writers.values():
        writer.release()

# Paths

csv_path = 'tracking_results.csv'  # Update this path
output_dir = 'output_videos'  # Update this path

# Make sure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Load tracking data and process the video
tracking_df = load_tracking_data(csv_path)
save_cropped_videos(video_path, tracking_df, output_dir)

print("Cropping complete. Videos saved in", output_dir)

Cropping complete. Videos saved in output_videos


: 

# crop by image

In [ ]:
from moviepy.editor import VideoFileClip
import numpy as np
import cv2
import os

# Define the function to extract and save frames based on tracking results
def extract_frames(video_path, tracking_data, output_folder):
    # Read the video file
    video = VideoFileClip(video_path)
    frame_rate = video.fps

    for index, row in tracking_data.iterrows():
        frame_number = row['Frame']
        box = eval(row['Box'])  # Convert string representation of list to actual list
        x1, y1, x2, y2 = box

        # Calculate height and width of the box
        height = y2 - y1
        width = x2 - x1

        # Calculate the size of the square to crop
        size = max(width, height)

        # Center the box
        x_center = x1 + width // 2
        y_center = y1 + height // 2

        # Calculate new corners of the square box
        new_x1 = max(x_center - size // 2, 0)
        new_y1 = max(y_center - size // 2, 0)
        new_x2 = min(x_center + size // 2, video.size[0])
        new_y2 = min(y_center + size // 2, video.size[1])

        # Extract the frame at the specified time
        time = frame_number / frame_rate
        frame = video.get_frame(time)

        # Crop the frame to the square
        cropped_frame = frame[new_y1:new_y2, new_x1:new_x2]

        # Save the cropped frame
        output_filename = os.path.join(output_folder, f"frame_{frame_number}.png")
        cv2.imwrite(output_filename, cv2.cvtColor(cropped_frame, cv2.COLOR_RGB2BGR))

    video.close()

# Define the paths
import pandas as pd
output_folder = 'cropped_frames'
tracking_results = pd.read_csv('tracking_results.csv')
os.makedirs(output_folder, exist_ok=True)

# Extract and save frames
extract_frames(video_path, tracking_results, output_folder)


# crop by image (cv2)

In [ ]:
import numpy as np
import cv2
import os
import pandas as pd

# Define the function to extract and save frames based on tracking results
def extract_frames(video_path, tracking_data, output_folder):
    # Open the video file
    video = cv2.VideoCapture(video_path)
    frame_rate = video.get(cv2.CAP_PROP_FPS)
    video_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    video_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

    for index, row in tracking_data.iterrows():
        frame_number = row['Frame']
        box = eval(row['Box'])  # Convert string representation of list to actual list
        x1, y1, x2, y2 = box

        # Calculate the maximum size of the square based on video dimensions
        max_square_size = min(video_width, video_height)

        # Calculate height and width of the box
        height = y2 - y1
        width = x2 - x1

        # Calculate the size of the square to crop, ensuring it does not exceed video dimensions
        size = min(max(width, height), max_square_size)

        # Center the box
        x_center = x1 + width // 2
        y_center = y1 + height // 2

        # Calculate new corners of the square box
        # Adjust the position to ensure the square does not exceed the video frame
        new_x1 = max(x_center - size // 2, 0)
        new_y1 = max(y_center - size // 2, 0)
        new_x2 = min(new_x1 + size, video_width)
        new_y2 = min(new_y1 + size, video_height)



        # Set the video frame position
        video.set(cv2.CAP_PROP_POS_FRAMES, frame_number)

        # Read the frame
        success, frame = video.read()
        if not success:
            continue  # Skip if the frame is not read successfully

        # Crop the frame to the square
        cropped_frame = frame[new_y1:new_y2, new_x1:new_x2]

        # Save the cropped frame
        output_filename = os.path.join(output_folder, f"frame_{frame_number}.png")
        cv2.imwrite(output_filename, cropped_frame)

    video.release()

# Define the paths
tracking_results = pd.read_csv('tracking_results.csv')
os.makedirs(output_folder, exist_ok=True)

# Extract and save frames
extract_frames(video_path, tracking_results, output_folder)


KeyboardInterrupt: 

# re crop video

In [ ]:
import cv2
import os
from moviepy.editor import VideoFileClip

# Function to extract frames based on ID and create videos
def extract_and_create_videos(video_path, tracking_data, output_folder):
    video = VideoFileClip(video_path)
    frame_rate = video.fps

    # Create a dictionary to store video writers for each ID
    video_writers = {}

    for index, row in tracking_data.iterrows():
        frame_number = row['Frame']
        box = eval(row['Box'])
        person_id = row['ID']
        x1, y1, x2, y2 = box

        # Calculate the maximum size of the square based on video dimensions
        max_square_size = min(video_width, video_height)

        # Calculate height and width of the box
        height = y2 - y1
        width = x2 - x1

        # Calculate the size of the square to crop, ensuring it does not exceed video dimensions
        size = min(max(width, height), max_square_size)

        # Center the box
        x_center = x1 + width // 2
        y_center = y1 + height // 2

        # Calculate new corners of the square box
        # Adjust the position to ensure the square does not exceed the video frame
        new_x1 = max(x_center - size // 2, 0)
        new_y1 = max(y_center - size // 2, 0)
        new_x2 = min(new_x1 + size, video_width)
        new_y2 = min(new_y1 + size, video_height)


        # Extract the frame at the specified time
        time = frame_number / frame_rate
        frame = video.get_frame(time)

        # Crop the frame to the square
        cropped_frame = frame[new_y1:new_y2, new_x1:new_x2]

        # Check if the video writer for this ID exists
        if person_id not in video_writers:
            output_filename = os.path.join(output_folder, f"video_id_{person_id}.mp4")
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            video_writers[person_id] = cv2.VideoWriter(output_filename, fourcc, frame_rate, (size, size))

        # Write the frame to the video
        video_writers[person_id].write(cv2.cvtColor(cropped_frame, cv2.COLOR_RGB2BGR))

    # Release all video writers
    for writer in video_writers.values():
        writer.release()

    video.close()

# Define the output folder for the videos
output_folder = 'cropped_videos'
os.makedirs(output_folder, exist_ok=True)

# Extract frames and create videos
extract_and_create_videos(video_path, tracking_results, output_folder)


# again! 300x300

In [3]:
import cv2
import os
import pandas as pd

# Define the function to extract and save frames based on tracking results
def extract_frames(video_path, tracking_data, output_folder):
    # Open the video file
    video = cv2.VideoCapture(video_path)
    frame_rate = video.get(cv2.CAP_PROP_FPS)
    video_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    video_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

    for index, row in tracking_data.iterrows():
        frame_number = row['Frame']
        box = eval(row['Box'])  # Convert string representation of list to actual list
        x1, y1, x2, y2 = box

        # Calculate the maximum size of the square based on video dimensions
        max_square_size = min(video_width, video_height)

        # Calculate height and width of the box
        height = y2 - y1
        width = x2 - x1

        # Calculate the size of the square to crop, ensuring it does not exceed video dimensions
        size = min(max(width, height), max_square_size)

        # Center the box
        x_center = x1 + width // 2
        y_center = y1 + height // 2

        # Calculate new corners of the square box
        # Adjust the position to ensure the square does not exceed the video frame
        new_x1 = max(x_center - size // 2, 0)
        new_y1 = max(y_center - size // 2, 0)
        new_x2 = min(new_x1 + size, video_width)
        new_y2 = min(new_y1 + size, video_height)

        # Set the video frame position
        video.set(cv2.CAP_PROP_POS_FRAMES, frame_number)

        # Read the frame
        success, frame = video.read()
        if not success:
            continue  # Skip if the frame is not read successfully

        # Crop the frame to the square
        cropped_frame = frame[new_y1:new_y2, new_x1:new_x2]

        # Resize the cropped frame to 300x300 pixels
        resized_frame = cv2.resize(cropped_frame, (300, 300))

        # Save the resized frame
        output_filename = os.path.join(output_folder, f"frame_{frame_number}.png")
        cv2.imwrite(output_filename, resized_frame)

    video.release()

# Define the paths
output_folder = 'cropped_videos'
video_path = '../data/slip.mp4'  # Replace with your video path
tracking_results = pd.read_csv('tracking_results.csv')
os.makedirs(output_folder, exist_ok=True)

# Extract and save frames
extract_frames(video_path, tracking_results, output_folder)


In [ ]:
import cv2
import os
from moviepy.editor import VideoFileClip

def extract_and_create_videos(video_path, tracking_data, output_folder):
    video = VideoFileClip(video_path)
    frame_rate = video.fps

    # Create a dictionary to store video writers for each ID
    video_writers = {}

    for index, row in tracking_data.iterrows():
        frame_number = row['Frame']
        box = eval(row['Box'])
        person_id = row['ID']
        x1, y1, x2, y2 = box

        # Calculate height and width of the box
        height = y2 - y1

        # Center the box
        x_center = x1 + (x2 - x1) // 2
        y_center = y1 + height // 2

        # Calculate new corners of the square box
        new_x1 = max(x_center - height // 2, 0)
        new_y1 = max(y_center - height // 2, 0)
        new_x2 = min(x_center + height // 2, video.size[0])
        new_y2 = min(y_center + height // 2, video.size[1])

        # Extract the frame at the specified time
        time = frame_number / frame_rate
        frame = video.get_frame(time)

        # Crop the frame to the square
        cropped_frame = frame[new_y1:new_y2, new_x1:new_x2]

        # Resize cropped frame to 300x300
        resized_frame = cv2.resize(cropped_frame, (300, 300))

        # Check if the video writer for this ID exists
        if person_id not in video_writers:
            output_filename = os.path.join(output_folder, f"video_id_{person_id}.mp4")
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            video_writers[person_id] = cv2.VideoWriter(output_filename, fourcc, frame_rate, (300, 300))

        # Write the frame to the video
        video_writers[person_id].write(cv2.cvtColor(resized_frame, cv2.COLOR_RGB2BGR))

    # Release all video writers
    for writer in video_writers.values():
        writer.release()

    video.close()

# Define the output folder for the videos
video_path = '../data/slip.mp4'  # Replace with your video path
tracking_results = pd.read_csv('tracking_results.csv')
output_folder = 'cropped_videos'
os.makedirs(output_folder, exist_ok=True)

# Extract frames and create videos
extract_and_create_videos(video_path, tracking_results, output_folder)


In [6]:
import cv2
import pandas as pd
import os

def extract_and_create_videos(video_path, tracking_data, output_folder):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise Exception("Error opening video file")

    # Get video properties
    frame_rate = cap.get(cv2.CAP_PROP_FPS)
    video_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    video_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Create a dictionary to store video writers for each ID
    video_writers = {}

    # Process each entry in the tracking data
    for index, row in tracking_data.iterrows():
        frame_number = row['Frame']
        box = eval(row['Box'])
        person_id = row['ID']
        x1, y1, x2, y2 = box

        # Ensure coordinates are within bounds
        x1, y1, x2, y2 = max(x1, 0), max(y1, 0), min(x2, video_width), min(y2, video_height)

        # Set the video to the specific frame
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
        ret, frame = cap.read()
        if not ret or frame is None:
            continue

        # Crop and resize frame
        cropped_frame = frame[y1:y2, x1:x2]
        if cropped_frame.size == 0:
            continue  # Skip if the cropped frame is empty
        resized_frame = cv2.resize(cropped_frame, (300, 300))

        # Check if the video writer for this ID exists
        if person_id not in video_writers:
            output_filename = os.path.join(output_folder, f"video_id_{person_id}.mp4")
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            video_writers[person_id] = cv2.VideoWriter(output_filename, fourcc, frame_rate, (300, 300))

        # Write the frame to the video
        video_writers[person_id].write(resized_frame)

    # Release all video writers
    for writer in video_writers.values():
        writer.release()

    cap.release()

# Define the output folder for the videos
video_path = 'mevid.mp4'  # Replace with your video path
tracking_data = pd.read_csv('abc.csv')
output_folder = 'cropped_videos'
os.makedirs(output_folder, exist_ok=True)

# Extract frames and create videos
extract_and_create_videos(video_path, tracking_data, output_folder)
